In [ ]:
# !pip install -q pdf2image ultralytics supervision

  You can safely remove it manually.
  You can safely remove it manually.


In [1]:
PDF_PATH = "../data/AR for improved learnability.pdf"

In [2]:
# Folder structure setup
import os

BASE_DIR = ""
DIRECTORIES = ["img/pages", "img/annotated", "img/figures"]
for dir in DIRECTORIES:
    os.makedirs(os.path.join(BASE_DIR, dir), exist_ok=True)

In [3]:
# handle image size
def resize_img(image):
    # change the image to RGB mode
    image = image.convert('RGB')

    # change image width to 1440 and maintain the aspect ratio
    basewidth = 1440
    wpercent = (basewidth / float(image.size[0]))

    # change the height of the image
    hsize = int((float(image.size[1]) * float(wpercent)))

    # resize the image
    image = image.resize((basewidth, hsize))

    return image

In [4]:
from pdf2image import convert_from_path

img_path = "img"
page_path = f"{img_path}/pages"


# Function to split PDF into images
def split_pdf(pdf_path):
    pages = []
    page_images = convert_from_path(pdf_path, 96)
    for i, page in enumerate(page_images):
        # Save Image
        # page = resize_img(page)
        path = f"{page_path}/page_{i}.png"
        page.save(path, "PNG")
        pages.append(path)

    return pages

pages = split_pdf(PDF_PATH)
pages

['img/pages/page_0.png',
 'img/pages/page_1.png',
 'img/pages/page_2.png',
 'img/pages/page_3.png',
 'img/pages/page_4.png',
 'img/pages/page_5.png',
 'img/pages/page_6.png',
 'img/pages/page_7.png',
 'img/pages/page_8.png']

In [6]:
import cv2
import numpy as np
from ultralytics import YOLO
import supervision as sv

fig_path = f"{img_path}/figures"

# detect text, table, and figure using YOLOv11 model
def detect_text(pages, model=YOLO("../model/best.pt")):
    detected_images = []
    annotated_images =[]
    figures = []
    for j, page in enumerate(pages):
        image = cv2.imread(page)
        results = model(image, conf=0.35, iou=0.7)[0]
        detections = sv.Detections.from_ultralytics(results)
        detected_images.append(detections)

        # save annotated image
        annotated_image = image.copy()
        annotated_image = sv.BoxAnnotator().annotate(scene=annotated_image, detections=detections)
        annotated_image = sv.LabelAnnotator().annotate(scene=annotated_image, detections=detections)
        output_image_path = f"img/annotated/page_{j+1}_annotated.png"
        cv2.imwrite(output_image_path, annotated_image)
        annotated_images.append(output_image_path)

        # save figure class_name
        # for i, class_name in enumerate(detections.data['class_name']):

        #     if class_name == 'figure':
        #         x1, y1, x2, y2 = map(int, detections.xyxy[i])
        #         section = image[y1:y2, x1:x2]
        #         output_filename = f"{fig_path}/page_{j}_figure_{i}.png"
        #         cv2.imwrite(output_filename, section)
        #         # save metadata
        #         metadata = {
        #             "file_path": output_filename,
        #             "bbox": [x1, y1, x2, y2],
        #             "name": "",
        #             "type": "",
        #             "data": "",
        #             "description": ""
        #         }
        #         page["figures"].append(metadata)
        #         figures.append(output_filename)

    return detected_images, annotated_images, figures

In [7]:
detected_images, annotated_images, figures = detect_text(pages)
detected_images[0]


0: 640x480 1 header, 1 image, 1 logo, 4 texts, 1 title, 1524.2ms
Speed: 92.6ms preprocess, 1524.2ms inference, 134.7ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 footer, 1 text, 139.6ms
Speed: 12.4ms preprocess, 139.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 footer, 1 table, 1 text, 120.4ms
Speed: 2.3ms preprocess, 120.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 footer, 1 header, 1 image, 2 logos, 1 text, 127.4ms
Speed: 2.6ms preprocess, 127.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 footer, 1 graph, 1 table, 2 texts, 136.7ms
Speed: 2.7ms preprocess, 136.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 footer, 3 tables, 2 texts, 1 title, 120.3ms
Speed: 2.3ms preprocess, 120.3ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 footer, 3 graphs, 1 table, 2 texts, 133.1ms
Speed: 2.6ms prep

Detections(xyxy=array([[     667.91,      193.03,      708.32,      239.29],
       [     46.813,      678.18,      745.85,      866.14],
       [     265.37,      350.59,       746.8,       647.2],
       [     235.46,      44.158,      558.88,      60.754],
       [     49.552,      217.73,      480.59,      243.81],
       [     45.289,      73.279,      132.54,      179.03],
       [     156.96,      78.929,      643.62,       174.1],
       [     46.033,      279.99,         573,      318.72]], dtype=float32), mask=None, confidence=array([    0.83712,     0.61782,     0.56934,     0.53512,     0.52899,     0.45109,     0.41553,     0.37539], dtype=float32), class_id=array([4, 7, 7, 2, 8, 3, 7, 7]), tracker_id=None, data={'class_name': array(['logo', 'text', 'text', 'header', 'title', 'image', 'text', 'text'], dtype='<U6')}, metadata={})